In [ ]:
import numpy as np
import os, sys, h5py
import open3d as o3d
import copy

In [ ]:
def drop_hole(pc, p):
    random_point = np.random.randint(0, pc.shape[0])
    index = np.linalg.norm(pc - pc[random_point].reshape(1,3), axis=1).argsort()
    return pc[index[int(pc.shape[0] * p):]]

In [ ]:

# Aug #6
def multi_drop_hole(pc, n, p):
    """input:
        pc = point cloud
        n = number of holes
        p = percentage of points per hole that need to be taken out to make the hole
        
        Future work: Maybe I can add a random=true condition if true will randomly add 1-5 holes everytime it's called in data augmentation
    """
#     p = p/n # this makes holes with a relative percentage. 
    n_points = pc.shape[0]*p
    for i in range(n):
        random_point = np.random.randint(0, pc.shape[0])
        index = np.linalg.norm(pc - pc[random_point].reshape(1,3), axis=1).argsort()
        pc = pc[index[int(pc.shape[0] * p):]]
        p = n_points/pc.shape[0] # recalculate p to take out fixed number of points from each hole
    return pc

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    # print(":: Downsample with a voxel size %.3f." % voxel_size)
    # pcd = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    # print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    # print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    pcd_fpfh = np.transpose(np.array(pcd_fpfh.data))
    return pcd_fpfh

In [ ]:
#train
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

#test
# root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
# h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)

In [ ]:
print('points start', points.shape)
# convert npy to pcd
out_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate_rotation_multi_patch_random_noise_4'

# local_features = np.empty([points.shape[0], points.shape[1], 33])
for i in range(points.shape[0]):

    # out_file_path = os.path.join(out_folder_path, str(i) + '_train' + ".pcd")
    out_file_path = os.path.join(out_folder_path, str(i) + '_test' + ".pcd")

    points_temp = copy.deepcopy(points[i])

    ## -------------------
    # Note:
    # All perturbations code are below.
    # You will find all the scale_ratio, p_drop and other values in the 'Results on Different Perturbations.xlsx' file that Dr. Rawat shared with you.
    ## -------------------


    ## -------------------
    ## random scale
    # scale_ratio = np.array([0.3, 0.4, 0.5])
    # points_temp *= scale_ratio

    ## -------------------
    ## scale
    # scale_ratio = 0.9
    # points_temp *= scale_ratio

    ## -------------------
    ## Drop one patch or make a hole in the point cloud. p_drop controls how big is the hole.
    # p_drop = 0.25
    # points_temp = drop_hole(pc=points_temp, p=p_drop)

    ## -------------------
    ## Add noise to point cloud. Change the value of p. The values can be found in the paper and the spreadsheet. 
    # p = 0.01
    # sigma = 0.1
    # N_, C_ = points_temp.shape

    # N__ = int(N_ * p)

    # # print('points_temp[:,0]', points_temp[:,0].shape)
    # x_min = np.min(points_temp[:,0])
    # x_max = np.max(points_temp[:,0])
    # y_min = np.min(points_temp[:,1])
    # y_max = np.max(points_temp[:,1])
    # z_min = np.min(points_temp[:,2])
    # z_max = np.max(points_temp[:,2])
    # noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    # points_temp = np.vstack((points_temp, noise_points))
    
    
    ## -------------------
    # # Rotation
    # rotation_angle = 90
    # cosval = np.cos(rotation_angle)
    # sinval = np.sin(rotation_angle)
    # rotation_matrix = np.array([[cosval, 0, sinval],
    #                             [0, 1, 0],
    #                             [-sinval, 0, cosval]])
    # R = rotation_matrix.astype(np.float32)
    # points_temp = np.matmul(points_temp, R)

    # drop one patch/portion
    # p_drop = 0.75
    # points_temp = drop_hole(pc=points_temp, p=p_drop)

    ### ---------------------------------
    ## Multiple holes in the PC. Chnage the value of n. We considered n is from 3 to 7. p stays same.
    # points_temp = multi_drop_hole(pc=points_temp, n=3, p=0.1)

    ### ---------------------------------
    # ## --------------------------------------
    # # Random points drop. 
    # p = 0.8
    # size = int(points_temp.shape[0] * (1-p))
    # random_indices = np.random.choice(points_temp.shape[0],
    #                                 size=size,
    #                                 replace=False)
    # points_temp = points_temp[random_indices, :]
    # # print('point_cloud after #3', point_cloud.shape)


    ### ---------------------------------
    # # # Jitter
    # sigma = 0.02
    # N_, C_ = points_temp.shape
    # clip = 0.2
    # points_temp += np.clip(sigma * np.random.randn(N_, C_), -1*clip, clip)

    ## ------------------------------
    # add cluster of random points 
    # num_cluster = 15
    # N__ = 200 #add 200 points in each cluster

    # cloud_v1 = o3d.geometry.PointCloud()
    # cloud_v1.points = o3d.utility.Vector3dVector(points_temp)

    # pcd_tree = o3d.geometry.KDTreeFlann(cloud_v1)

    # rand_idx_cluster = np.random.choice(points_temp.shape[0],
    #                                     size=num_cluster,
    #                                     replace=False)

    # for i in range(len(rand_idx_cluster)):
    #     [k, idx_tree, _] = pcd_tree.search_knn_vector_3d(cloud_v1.points[rand_idx_cluster[i]], 20)
    #     points_small = points_temp[idx_tree]

    #     # print('points_temp[:,0]', points_temp[:,0].shape)
    #     x_min = np.min(points_small[:,0])
    #     x_max = np.max(points_small[:,0])
    #     y_min = np.min(points_small[:,1])
    #     y_max = np.max(points_small[:,1])
    #     z_min = np.min(points_small[:,2])
    #     z_max = np.max(points_small[:,2])            

    #     noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    #     points_temp = np.vstack((points_temp, noise_points))




    ## ------------------------------
    # # # translate 1
    # translate_ratio = 1.0
    # points_temp += translate_ratio

    ## ------------------------------
    # # # # translate 2
    # translate_ratio = np.array([0,0,1.0])
    # points_temp += translate_ratio

    ## ------------------------------
    # # # # Rotation
    # rotation_angle = 90
    # pcd_temp = o3d.geometry.PointCloud()
    # pcd_temp.points = o3d.utility.Vector3dVector(points_temp)
    # R = pcd_temp.get_rotation_matrix_from_xyz((0, rotation_angle, 0))
    # R = R.astype(np.float32)
    # points_temp = np.matmul(points_temp, R)

    ## -------------------------

    # # # Add Random Points (Noise: inlier/outlier)
    p = 0.1
    sigma = 0.1
    N_, C_ = points_temp.shape

    N__ = int(N_ * p)

    # print('points_temp[:,0]', points_temp[:,0].shape)
    x_min = np.min(points_temp[:,0])
    x_max = np.max(points_temp[:,0])
    y_min = np.min(points_temp[:,1])
    y_max = np.max(points_temp[:,1])
    z_min = np.min(points_temp[:,2])
    z_max = np.max(points_temp[:,2])
    noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    points_temp = np.vstack((points_temp, noise_points))

    ## --------------------
    # keep total number of points 2048.
    if points_temp.shape[0] > points[i].shape[0]:
        rand_indices = np.random.choice(points_temp.shape[0],
                                        size=points[i].shape[0],
                                        replace=False)
        points_temp = points_temp[rand_indices, :]
    elif points_temp.shape[0] < points[i].shape[0]:
        rand_indices = np.random.choice(points_temp.shape[0],
                                        size=points[i].shape[0],
                                        replace=True)
        points_temp = points_temp[rand_indices, :]
    else:
        points_temp = points_temp

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_temp)

    o3d.io.write_point_cloud(out_file_path, pcd)
    
print('Done')